In [1]:
import pandas as pd
from datetime import datetime
import pandasql
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans

In [2]:
df = pd.read_csv("logins.txt.gz", compression='gzip', sep="\t", header=None)
df[["First Letter", "Name"]] = df[2].str.split(".", expand=True)

def normalize(seq):
    min_val = seq.min()
    max_val = seq.max()
    normalized_seq = (seq - min_val) / (max_val - min_val)
    return normalized_seq

df

,0,1,2,3,First Letter,Name
0,2021-01-01,00:00:00,s.matta,OUT,s,matta
1,2021-01-01,00:05:00,s.paige,OUT,s,paige
2,2021-01-01,01:52:00,t.goldschmidt,OUT,t,goldschmidt
3,2021-01-01,01:54:00,t.hardin,OUT,t,hardin
4,2021-01-01,02:40:00,c.elson,OUT,c,elson
...,...,...,...,...,...,...
3771147,2022-03-18,00:53:00,m.ponds,OUT,m,ponds
3771148,2022-03-18,16:54:00,m.ponds,IN,m,ponds
3771149,2022-03-19,00:57:00,m.ponds,OUT,m,ponds
3771150,2022-03-22,16:54:00,m.ponds,IN,m,ponds


In [3]:
def time2num(time: str):
    hours, minutes, seconds = map(int, time.split(":"))
    return hours * 3600 + minutes * 60 + seconds


try:
    df[1] = df[1].apply(time2num)
    # df[1] = normalize(df[1])
except:
    print("Already converted")


In [4]:
query3 = """
SELECT [2], AVG([1]) as AvgColumn
FROM df
WHERE [3] = 'IN'
GROUP BY [2]
ORDER BY AVG([1]) DESC
"""

result = pandasql.sqldf(query3, locals())

In [5]:
result = result.rename(columns={'2':'username'})
df = df.rename(columns={2:'username'})
df

,0,1,username,3,First Letter,Name
0,2021-01-01,0,s.matta,OUT,s,matta
1,2021-01-01,300,s.paige,OUT,s,paige
2,2021-01-01,6720,t.goldschmidt,OUT,t,goldschmidt
3,2021-01-01,6840,t.hardin,OUT,t,hardin
4,2021-01-01,9600,c.elson,OUT,c,elson
...,...,...,...,...,...,...
3771147,2022-03-18,3180,m.ponds,OUT,m,ponds
3771148,2022-03-18,60840,m.ponds,IN,m,ponds
3771149,2022-03-19,3420,m.ponds,OUT,m,ponds
3771150,2022-03-22,60840,m.ponds,IN,m,ponds


In [6]:
result1 = pd.merge(df[df[3]=='IN'], result, on='username')
result1['timing']=result1[1]-result1['AvgColumn']
#result1['lateness'] = df.groupby('username')['timing'].sum()
result1

,0,1,username,3,First Letter,Name,AvgColumn,timing
0,2021-01-01,21420,l.wood,IN,l,wood,21513.786408,-93.786408
1,2021-01-02,21780,l.wood,IN,l,wood,21513.786408,266.213592
2,2021-01-03,21360,l.wood,IN,l,wood,21513.786408,-153.786408
3,2021-01-04,21240,l.wood,IN,l,wood,21513.786408,-273.786408
4,2021-01-05,21600,l.wood,IN,l,wood,21513.786408,86.213592
...,...,...,...,...,...,...,...,...
1885571,2021-12-29,57660,l.bowens,IN,l,bowens,57733.333333,-73.333333
1885572,2021-12-30,57300,l.bowens,IN,l,bowens,57733.333333,-433.333333
1885573,2021-12-31,57480,l.bowens,IN,l,bowens,57733.333333,-253.333333
1885574,2021-12-27,25080,n.schaller,IN,n,schaller,25290.000000,-210.000000


In [26]:
# Most popular days:
days = result1[0].value_counts()[:30].index.tolist()

In [35]:
daysDF = result1[result1[0].isin(days)]
daysDF

,0,1,username,3,First Letter,Name,AvgColumn,timing
1,2021-01-02,21780,l.wood,IN,l,wood,21513.786408,266.213592
2,2021-01-03,21360,l.wood,IN,l,wood,21513.786408,-153.786408
3,2021-01-04,21240,l.wood,IN,l,wood,21513.786408,-273.786408
4,2021-01-05,21600,l.wood,IN,l,wood,21513.786408,86.213592
5,2021-01-06,21360,l.wood,IN,l,wood,21513.786408,-153.786408
...,...,...,...,...,...,...,...,...
1883058,2021-05-24,63300,j.saunders,IN,j,saunders,63224.140969,75.859031
1883276,2021-05-24,59340,j.155,IN,j,155,59084.477612,255.522388
1883469,2021-05-24,28440,c.56,IN,c,56,28581.900000,-141.900000
1883663,2021-05-24,27360,v.sinclair,IN,v,sinclair,27105.164835,254.835165


In [38]:
daysDF=daysDF.drop_duplicates(subset=[0, 'username'], keep='first')

In [39]:
daysLong = daysDF.pivot(index='username', columns=0, values='timing')
daysLong

,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-10,2021-01-11,2021-01-16,2021-01-17,...,2021-02-21,2021-02-22,2021-02-23,2021-02-28,2021-03-02,2021-03-14,2021-03-16,2021-03-23,2021-04-10,2021-05-24
username,,,,,,,,,,,,,,,,,,,,,
a.1,114.276316,-185.723684,294.276316,-245.723684,-245.723684,-245.723684,-305.723684,NaN,-305.723684,-305.723684,...,-185.723684,-305.723684,114.276316,-305.723684,-65.723684,-5.723684,NaN,NaN,-365.723684,-245.723684
a.10,159.619048,-20.380952,-260.380952,-80.380952,39.619048,-320.380952,39.619048,-20.380952,159.619048,-140.380952,...,-200.380952,-380.380952,-320.380952,159.619048,99.619048,99.619048,NaN,459.619048,-260.380952,-20.380952
a.11,NaN,-285.732899,-165.732899,-405.732899,134.267101,NaN,NaN,NaN,-225.732899,134.267101,...,134.267101,74.267101,-405.732899,NaN,14.267101,-225.732899,74.267101,374.267101,74.267101,-405.732899
a.12,-45.810811,74.189189,-105.810811,-345.810811,134.189189,-165.810811,-165.810811,-225.810811,14.189189,NaN,...,-225.810811,-165.810811,74.189189,-165.810811,-165.810811,NaN,-45.810811,NaN,NaN,NaN
a.13,-337.748344,82.251656,NaN,NaN,-157.748344,-397.748344,-217.748344,-97.748344,-157.748344,-157.748344,...,NaN,-337.748344,-337.748344,142.251656,NaN,-157.748344,-397.748344,1882.251656,-277.748344,-337.748344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
z.norman,-162.095238,197.904762,497.904762,-342.095238,NaN,-282.095238,-222.095238,137.904762,77.904762,NaN,...,137.904762,-282.095238,-42.095238,NaN,-102.095238,-342.095238,77.904762,17.904762,-222.095238,-162.095238
z.shackley,NaN,NaN,NaN,-415.548387,4.451613,4.451613,-175.548387,-415.548387,NaN,NaN,...,-295.548387,124.451613,-55.548387,-295.548387,-235.548387,64.451613,-235.548387,1324.451613,-55.548387,-235.548387
z.sommers,NaN,-36.078431,143.921569,203.921569,23.921569,-96.078431,23.921569,23.921569,NaN,83.921569,...,-276.078431,-36.078431,263.921569,203.921569,-276.078431,NaN,-216.078431,-96.078431,-216.078431,-96.078431


In [42]:
daysLong[daysLong.index=='a.1']

,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-10,2021-01-11,2021-01-16,2021-01-17,...,2021-02-21,2021-02-22,2021-02-23,2021-02-28,2021-03-02,2021-03-14,2021-03-16,2021-03-23,2021-04-10,2021-05-24
username,,,,,,,,,,,,,,,,,,,,,
a.1,114.276316,-185.723684,294.276316,-245.723684,-245.723684,-245.723684,-305.723684,NaN,-305.723684,-305.723684,...,-185.723684,-305.723684,114.276316,-305.723684,-65.723684,-5.723684,NaN,NaN,-365.723684,-245.723684


In [44]:
for col in daysLong.columns:
    daysLong[col].fillna((daysLong[col].mean()), inplace=True)

In [70]:
kmeans = KMeans(n_clusters=5, init='random')
kmeans.fit(daysLong)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(init='random', n_clusters=5)

In [71]:
daysLong['cluster'] = kmeans.predict(daysLong)
centroids = kmeans.cluster_centers_

In [72]:
west = daysLong[daysLong.index=='s.kinkel']['cluster'][0]
daysLong[daysLong['cluster']==west]

,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-10,2021-01-11,2021-01-16,2021-01-17,...,2021-02-22,2021-02-23,2021-02-28,2021-03-02,2021-03-14,2021-03-16,2021-03-23,2021-04-10,2021-05-24,cluster
username,,,,,,,,,,,,,,,,,,,,,
a.1,114.276316,-185.723684,294.276316,-245.723684,-245.723684,-245.723684,-305.723684,14.632121,-305.723684,-305.723684,...,-305.723684,114.276316,-305.723684,-65.723684,-5.723684,-92.133323,327.092047,-365.723684,-245.723684,0
a.12,-45.810811,74.189189,-105.810811,-345.810811,134.189189,-165.810811,-165.810811,-225.810811,14.189189,-83.658881,...,-165.810811,74.189189,-165.810811,-165.810811,-120.386054,-45.810811,327.092047,-75.346528,-99.584826,0
a.2,49.342105,-70.657895,1669.342105,-190.657895,-130.657895,-310.657895,-101.062110,-10.657895,-310.657895,-130.657895,...,-190.657895,-108.548223,-70.657895,-190.657895,49.342105,-70.657895,109.342105,-310.657895,-310.657895,0
a.25,-265.552050,94.447950,-145.552050,-145.552050,-25.552050,94.447950,-145.552050,214.447950,-205.552050,34.447950,...,-110.751807,-108.548223,-145.552050,-205.552050,-120.386054,-92.133323,-265.552050,214.447950,94.447950,0
a.26,-90.729792,-81.788203,134.037964,-144.466019,-264.466019,-204.466019,-144.466019,215.533981,155.533981,-24.466019,...,155.533981,-264.466019,35.533981,215.533981,-24.466019,-92.133323,-24.466019,-24.466019,-204.466019,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
z.mcdonald,-90.729792,268.641115,-31.358885,88.641115,-151.358885,-31.358885,-101.062110,-91.358885,-31.358885,268.641115,...,268.641115,-31.358885,208.641115,208.641115,-120.386054,-92.133323,-271.358885,-31.358885,-99.584826,0
z.norman,-162.095238,197.904762,497.904762,-342.095238,-83.109786,-282.095238,-222.095238,137.904762,77.904762,-83.658881,...,-282.095238,-42.095238,-102.226783,-102.095238,-342.095238,77.904762,17.904762,-222.095238,-162.095238,0
z.sommers,-90.729792,-36.078431,143.921569,203.921569,23.921569,-96.078431,23.921569,23.921569,-98.898009,83.921569,...,-36.078431,263.921569,203.921569,-276.078431,-120.386054,-216.078431,-96.078431,-216.078431,-96.078431,0


In [74]:
daysNorm = daysLong.div(daysLong.abs().max(axis=1), axis=0)
daysNorm

,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-10,2021-01-11,2021-01-16,2021-01-17,...,2021-02-22,2021-02-23,2021-02-28,2021-03-02,2021-03-14,2021-03-16,2021-03-23,2021-04-10,2021-05-24,cluster
username,,,,,,,,,,,,,,,,,,,,,
a.1,0.312466,-0.507825,0.804641,-0.671883,-0.671883,-0.671883,-0.835942,0.040009,-0.835942,-0.835942,...,-0.835942,0.312466,-0.835942,-0.179709,-0.015650,-0.251921,0.894369,-1.000000,-0.671883,0.000000
a.10,0.135314,-0.017278,-0.220733,-0.068141,0.033586,-0.271597,0.033586,-0.017278,0.135314,-0.119005,...,-0.322461,-0.271597,0.135314,0.084450,0.084450,-0.078104,0.389633,-0.220733,-0.017278,0.001695
a.11,-0.055517,-0.174839,-0.101411,-0.248266,0.082157,-0.056521,-0.061839,0.008953,-0.138125,0.082157,...,0.045444,-0.248266,-0.062552,0.008730,-0.138125,0.045444,0.229012,0.045444,-0.248266,0.001224
a.12,-0.062396,0.101049,-0.144119,-0.471010,0.182772,-0.225842,-0.225842,-0.307565,0.019326,-0.113947,...,-0.225842,0.101049,-0.225842,-0.225842,-0.163971,-0.062396,0.445515,-0.102625,-0.135639,0.000000
a.13,-0.179438,0.043699,0.071212,0.014756,-0.083808,-0.211315,-0.115685,-0.051932,-0.083808,-0.083808,...,-0.179438,-0.179438,0.075575,-0.050861,-0.083808,-0.211315,1.000000,-0.147562,-0.179438,0.001063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
z.norman,-0.325555,0.397475,1.000000,-0.687070,-0.166919,-0.566565,-0.446060,0.276970,0.156465,-0.168022,...,-0.566565,-0.084545,-0.205314,-0.205050,-0.687070,0.156465,0.035960,-0.446060,-0.325555,0.000000
z.shackley,-0.068504,-0.061753,0.101203,-0.313751,0.003361,0.003361,-0.132544,-0.313751,-0.074671,-0.063165,...,0.093965,-0.041941,-0.223148,-0.177846,0.048663,-0.177846,1.000000,-0.041941,-0.177846,0.001510
z.sommers,-0.328638,-0.130682,0.521307,0.738636,0.086648,-0.348011,0.086648,0.086648,-0.358224,0.303977,...,-0.130682,0.955966,0.738636,-1.000000,-0.436057,-0.782670,-0.348011,-0.782670,-0.348011,0.000000


In [78]:
kmeans = KMeans(n_clusters=5, init='k-means++')
kmeans.fit(daysNorm)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=5)

In [79]:
daysNorm['cluster'] = kmeans.predict(daysNorm)
centroids = kmeans.cluster_centers_

In [84]:
west = daysNorm[daysNorm.index=='s.kinkel']['cluster'][0]
westerners = daysNorm[daysNorm['cluster']==west].index

In [82]:
daysNorm['cluster'].value_counts()

cluster
1    1993
2    1635
4    1192
3     859
0     582
Name: count, dtype: int64

In [87]:
westDF = result1[result1['username'].isin(westerners)]
westDF

,0,1,username,3,First Letter,Name,AvgColumn,timing
2124,2021-01-01,23280,s.hewett,IN,s,hewett,23185.563380,94.436620
2125,2021-01-02,23160,s.hewett,IN,s,hewett,23185.563380,-25.563380
2126,2021-01-03,23400,s.hewett,IN,s,hewett,23185.563380,214.436620
2127,2021-01-05,23100,s.hewett,IN,s,hewett,23185.563380,-85.563380
2128,2021-01-06,23160,s.hewett,IN,s,hewett,23185.563380,-25.563380
...,...,...,...,...,...,...,...,...
1880238,2021-12-27,32400,v.lattea,IN,v,lattea,32306.384365,93.615635
1880239,2021-12-28,32280,v.lattea,IN,v,lattea,32306.384365,-26.384365
1880240,2021-12-29,32100,v.lattea,IN,v,lattea,32306.384365,-206.384365
1880241,2021-12-30,32520,v.lattea,IN,v,lattea,32306.384365,213.615635


In [95]:
westDF.groupby(0)['timing'].mean().sort_values(ascending=False).head(20)

0
2021-09-16    1457.993421
2021-09-17    1444.519806
2021-07-23    1436.745430
2021-12-17    1428.657327
2021-08-25    1412.999625
2021-12-15    1364.550097
2021-11-25     849.620106
2021-01-11     735.296366
2021-01-21     705.155963
2021-04-01     684.554250
2021-09-24     654.653039
2021-09-10     650.037055
2021-12-09     641.924901
2021-06-04     640.367528
2021-03-18     636.460020
2021-04-15     625.352923
2021-10-11     622.002463
2021-06-14     610.824698
2021-11-23     609.734523
2021-06-08     607.086666
Name: timing, dtype: float64